In [6]:
import sys
import os  
from datetime import datetime, timedelta, date
import datetime as dt

import requests
import pandas as pd
import json
import yaml
import importlib


from snowflake.snowpark.functions import date_trunc, current_date, to_date, month
from snowflake.snowpark.functions import col, lit, when
from snowflake.snowpark.types import DateType
import snowflake.snowpark.functions as f


sys.path.append(os.path.abspath('scripts'))

try:
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
except Exception:
    import SnowflakeConnector  # brings module into scope
    importlib.reload(SnowflakeConnector)
    from SnowflakeConnector import create_active_session
    session = create_active_session()


import utils
importlib.reload(utils)
from utils import get_orders_for_date, get_max_date_from_table


In [3]:
#Access Token
TOKEN_PATH = "config/shopify_auth.yaml"
QUERY_PATH = "graphql/queries/get_transactions.graphql"
SELLINGPLAN_TABLE = "snowpark_db.shopify.orders_with_selling_plan"
DATE_COLUMN = "PROCESSED_AT"


with open(TOKEN_PATH, "r") as file:
    config = yaml.safe_load(file)
ACCESS_TOKEN = config.get("ACCESS_TOKEN", {})


with open(QUERY_PATH, "r") as file:
    QUERY_STRING = file.read()


SHOP_ENDPOINT = "https://shopmbg.myshopify.com/admin/api/2024-04/graphql.json"


In [ ]:
# ##See  SELLINGPLAN_TABLE daily summary

# SELLINGPLAN = session.table(SELLINGPLAN_TABLE)\
#     .select(to_date(col(DATE_COLUMN)).alias("PROCESSED_DATE"), 
#             col("ORDER_ID"), 
#             col("ORDER_NAME"), 
#             col("LINEITEM_ID"), 
#             col("TAGS"), 
#             col("LOAD_TIMESTAMP")).group_by(col("PROCESSED_DATE"))\
#     .agg(f.count(col("ORDER_ID")).alias("ORDER_COUNT")).sort(col("PROCESSED_DATE").desc())

# SELLINGPLAN.show()


In [ ]:
##See  SELLINGPLAN_TABLE monthly summary
# SELLINGPLAN = session.table(SELLINGPLAN_TABLE)\
#     .select(month(to_date(col(DATE_COLUMN))).alias("PROCESSED_MONTH"), 
#             col("ORDER_ID"), 
#             col("ORDER_NAME"), 
#             col("LINEITEM_ID"), 
#             col("TAGS"), 
#             col("LOAD_TIMESTAMP")).group_by(col("PROCESSED_MONTH"))\
#     .agg(f.count(col("ORDER_ID")).alias("ORDER_COUNT")).sort(col("PROCESSED_MONTH").desc())

# SELLINGPLAN.show()


In [ ]:
orders = get_orders_for_date("2025-06-11", ACCESS_TOKEN, SHOP_ENDPOINT, QUERY_STRING)


In [11]:
#maxdate in SELLINGPLAN_TABLE

max_date_in_table = get_max_date_from_table(SELLINGPLAN_TABLE, DATE_COLUMN, session)
max_date_in_table.strftime("%Y-%m-%d")

'2025-06-16'

In [12]:

load_date = max_date_in_table + timedelta(days=1)
load_date = load_date.strftime("%Y-%m-%d")


all_orders = get_orders_for_date(load_date, ACCESS_TOKEN, SHOP_ENDPOINT, QUERY_STRING)
now = datetime.utcnow()
paginated_df = pd.DataFrame([
        {
            "PROCESSED_AT" :o['node']['processedAt'] ,
            "ORDER_ID": o['node']['id'].split('/')[-1],
            "ORDER_NAME": o['node']['name'].split('#')[-1],
            "LINEITEM_ID":json.dumps(o['node']['lineItems']),
            "TAGS":json.dumps(o['node']['tags']),
            "LOAD_TIMESTAMP": now
        }
        for o in all_orders
    ])

end = datetime.strptime(date, "%Y-%m-%d") + timedelta(days=1)
end = end.strftime('%Y-%m-%dT00:00:00Z')
paginated_df = paginated_df[paginated_df["PROCESSED_AT"]<end]


print(f"end = {end}")


paginated_df.tail(10)

#1769


Fetched 1006 orders for 2025-06-17


KeyError: 'processedAt'

In [ ]:
#final extraction  (DAILY)
#date_list = date_generator(stop_date=date(2025, 1, 1))


all_orders = get_orders_for_date(next_date)
now = datetime.utcnow()
paginated_df = pd.DataFrame([
    {
        "PROCESSED_AT" :o['node']['processedAt'] ,
        "ORDER_ID": o['node']['id'].split('/')[-1],
        "ORDER_NAME": o['node']['name'].split('#')[-1],
        "LINEITEM_ID":json.dumps(o['node']['lineItems']),
        "TAGS":json.dumps(o['node']['tags']),
        "LOAD_TIMESTAMP": now
    }
    for o in all_orders
])

#
end = datetime.strptime(date, "%Y-%m-%d") + timedelta(days=1)
end = end.strftime('%Y-%m-%dT00:00:00Z')
paginated_df = paginated_df[paginated_df["PROCESSED_AT"]<end]


records = paginated_df.to_dict(orient="records")
transactions = session.create_dataframe(records)
transactions.write.save_as_table("snowpark_db.shopify.orders_with_selling_plan", mode="append")


print(f"Updated transactions for {next_date}")
